# Setup

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean
import random
import numpy as np
from copy import deepcopy
import itertools
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings, get_cell_filepaths

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


### Import settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [2]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

In [3]:
dirpath = Path(settings_dict['save_dir'])
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

## Get all cells info

In [57]:
nodefiles = get_cell_filepaths(dirpath)

In [58]:
cell_type = {}
not_typed = []
for x in nodefiles.keys():
    cell = ecrest(settings_dict,filepath = nodefiles[x],launch_viewer=False)
    cell_type[x] = cell.get_ctype('manual') 
    if (cell.get_ctype('manual') == []) | (cell.get_ctype('manual') == ''):
        cell_type[x]='none'
        not_typed.append(x)# print(f'cell {x} is not cell-typed in json')
        
print('the following cells are not typed in the main network')
print(not_typed)        
        

the following cells are not typed in the main network
['139151621', '223880142']


In [21]:
set([k for k,v in cell_type.items() if v in 'grc'])

set()

## Get all segments

In [59]:
# Create a base_segments dictionary of all reconstructed cells 

base_segments = {}
for x,f in nodefiles.items():
    # if cell_type[x] in network_types: # if do this, you can't check if the post-syn segments exist as a reconstruction
    cell = ecrest(settings_dict,filepath = f)#,launch_viewer=False)
    base_segments[cell.cell_data['metadata']['main_seg']['base']] = cell.cell_data['base_segments']
    
    try:
        assert cell.cell_data['metadata']['main_seg']['base'] == x
    except:
        print(x,cell.cell_data['metadata']['main_seg']['base'])

## Cell structure labeling checks

In [ ]:
print('need labeling for:')

for x,segs in base_segments.items():
    if (len(segs['unknown']) == len([s for k,v in segs.items() for s in v])) & (cell_type[x] in ['lf','lg']):
        print(f'{x} {cell_type[x]}')

## Build Graph

In [60]:
network_types = set([v for v in cell_type.values()])#['mg1','mg2','lg','lf','sg1','sg2','sgx','grc','aff']

In [61]:
synanno_type = 'post-synaptic'
vx_sizes = [16, 16, 30]

## find edges and set the cell-structure attribute of the edge based on which part of the cell the edge goes to
edge_list = []

with tqdm(total=len(nodefiles.keys())) as pbar:
    for x_pre in nodefiles.keys():
        pbar.update(1)
        if cell_type[x_pre] in network_types:
            
            # if the node has post-synaptic annotations (the current cell is assumed pre-synaptic)
            pre = ecrest(settings_dict,filepath = nodefiles[x_pre])#,launch_viewer=False)
            if pre.cell_data['end_points'][synanno_type] != []:
                # for each synapse
                for syn_ in pre.cell_data['end_points'][synanno_type]:
                    try:
                        post_seg = syn_[3]
                        syn_ = array([int(syn_[i]) for i in range(3)]) # synapses annotations exported as nanometers, so do not need to convert

                        # go through each other nodes
                        for x_post in nodefiles.keys():
                            if cell_type[x_post] in network_types:
                                post = base_segments[x_post] 
                                for k,v in post.items():
                                    for v_ in list(v): #find keys (can be multiple on the same cell) for matching segment ids
                                        if post_seg == v_: 
                                            # add edge to the graph between current node and matching node
                                            
                                            edge_list.append([x_pre,x_post,k,syn_[0],syn_[1],syn_[2]])
                                            

                    except IndexError as msg:
                        cellid = x_pre
                        print(msg, f'for cell {cellid} synapse at {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])} voxels has no segment id')

            else:
                continue


100%|███████████████████████████████████████| 2624/2624 [24:33<00:00,  1.78it/s]


### Specific cell(S)

In [52]:
edge_list = []
cell_list = ['290280494']
synanno_type = 'post-synaptic'

for x_pre in cell_list:
    pre = ecrest(settings_dict,filepath = nodefiles[x_pre])
    for syn_ in pre.cell_data['end_points'][synanno_type]:
        try:
            post_seg = syn_[3]
            syn_ = array([int(syn_[i]/1000) for i in range(3)])

            # go through each other nodes
            for x_post in nodefiles.keys():
                if cell_type[x_post] in network_types:
                    post = base_segments[x_post] 
                    for k,v in post.items():
                        for v_ in list(v): #find keys (can be multiple on the same cell) for matching segment ids
                            if post_seg == v_: 
                                # add edge to the graph between current node and matching node

                                edge_list.append([x_pre,x_post,k,syn_[0],syn_[1],syn_[2]])

        except IndexError as msg:
            cellid = x_pre
            print(msg, f'for cell {cellid} synapse at {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])} voxels has no segment id')




### Synapses dataframe

In [62]:
df_syn = pd.DataFrame(edge_list,columns = ['pre','post','structure','x','y','z'])

for i,r in df_syn.iterrows():
    df_syn.loc[i,'pre_type']=cell_type[df_syn.loc[i,'pre']]
    df_syn.loc[i,'post_type']=cell_type[df_syn.loc[i,'post']]

#### If want to peak at df_Edges

In [54]:
df_edges = df_syn[['pre','post','pre_type','post_type']].value_counts().reset_index(name='weight') #'structure',

In [55]:
df_edges#[df_edges['post_type'].isin(['mg2','lg'])]

,pre,post,pre_type,post_type,weight
0,290280494,127672247,mli,sg2,11
1,290280494,568416503,mli,uk,5
2,290280494,483687142,mli,lf,5
3,290280494,645210795,mli,lg,4
4,290280494,644175434,mli,fov,4
...,...,...,...,...,...
81,290280494,387554847,mli,sg1,1
82,290280494,387677355,mli,sg1,1
83,290280494,387708054,mli,sg1,1
84,290280494,387710569,mli,sg1,1


In [56]:
df_edges[['pre','post_type']].groupby('post_type')['pre'].count()

post_type
dml     8
fov     7
lf      5
lg      4
mg1    11
mg2     8
sg1    22
sg2     2
tsd     7
uk     12
Name: pre, dtype: int64

In [19]:
for i,r in df_edges.iterrows():
    df_edges.loc[i,'pre_diam']=soma_diam[str(df_edges.loc[i,'pre'])]
    df_edges.loc[i,'post_diam']=soma_diam[str(df_edges.loc[i,'post'])]

df_edges.loc[:,'diam_diff'] = (df_edges['post_diam']-df_edges['pre_diam'])/df_edges['pre_diam']

In [ ]:
# df_edges

focal_cell_id = df_edges['post'].unique()
display(df_syn[df_syn['pre']==focal_cell_id][['post','post_type']].value_counts().reset_index(
    name='weight')['post_type'].value_counts().reset_index(name='ncells'))

### save df_syn

In [63]:
savepath = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/graphs/')

df_syn.to_csv(savepath / 'df_postsyn.csv')

In [64]:
df_syn

,pre,post,structure,x,y,z,pre_type,post_type
0,126618353,387850816,basal dendrite,369584,313776,52260,mg1,mg2
1,126618353,303212610,basal dendrite,368736,314784,52560,mg1,mg2
2,126618353,394686712,unknown,370112,314832,52620,mg1,lf
3,126618353,394686712,basal dendrite,359792,314608,51390,mg1,lf
4,126618353,213605530,basal dendrite,360096,313808,51390,mg1,mg2
...,...,...,...,...,...,...,...,...
9522,648567066,645162652,basal dendrite,222320,293280,92280,sg2,lg
9523,648567066,645162652,basal dendrite,225152,290064,92310,sg2,lg
9524,648567066,559258918,basal dendrite,225792,289984,92310,sg2,sg1
9525,648567066,645162652,basal dendrite,225360,287248,92130,sg2,lg
